You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361307884"> Bond, Z-Scores and Scatterplots (15 min)</a>

In [1]:
import bql
import bqplot as bqp
import pandas as pd

bq = bql.Service()

In [2]:
Members = bq.univ.members('LUACTRUU Index')

dates = bq.func.range('2019-12-31','2020-04-10')

spread = bq.func.dropna(bq.data.spread(dates=dates))
current_spread = bq.func.last(spread)

z_score = bq.func.z_score(spread)
current_z_score = bq.func.last(z_score)

fields = {'Current Spread': current_spread, 'Current Z Score': current_z_score}

rank = bq.func.grouprank(current_z_score)

Filtered_Members = bq.univ.filter(Members,rank<=200)

request = bql.Request(Filtered_Members,fields)
response = bq.execute(request)

dataframe = bql.combined_df(response)

In [3]:
dataframe.reset_index(inplace=True)

dataframe.head()

,ID,DATE,Current Spread,Current Z Score
0,AN267754 Corp,2020-04-09,235.445679,2.197980
1,EH502090 Corp,2020-04-09,295.915192,2.326671
2,EI080338 Corp,2020-04-09,282.963989,2.130137
3,EJ409739 Corp,2020-04-09,422.861877,2.670720
4,AT725884 Corp,2020-04-09,221.979004,3.126435


In [4]:
import bqplot as bqp
from bqplot.interacts import BrushSelector
import pandas as pd
import numpy as np
from ipywidgets import Dropdown, HBox, VBox, HTML
import bqwidgets as bqw


# Create scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create marks
mark_scatter = bqp.Scatter(x=dataframe[dataframe.columns[2]],
                           y=dataframe[dataframe.columns[3]],
                           scales={'x': scale_x, 'y': scale_y},
                           default_size=48,
                           colors=['#1B84ED'])

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label=dataframe.columns[2])
axis_y = bqp.Axis(scale=scale_y,
                  orientation='vertical',
                  tick_format='0.0f',
                  label=dataframe.columns[3])

# Create selector
selector = BrushSelector(x_scale=scale_x,
                         y_scale=scale_y,
                         marks=[mark_scatter])

# Create Figure
figure = bqp.Figure(marks=[mark_scatter],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    layout={'width':'99%', 'height':'400px'},
                    padding_x=0.05,
                    title='Interactive scatter plot with linked datagrid',
                    title_style={'font-size': '22px'},
                    padding_y=0.05,
                    interaction=selector,
                    fig_margin={'top': 50, 'bottom': 60,
                                'left': 50, 'right':30})

# Create dropown widgets
dropdown_x = Dropdown(description='X axis',
                      options=dataframe.columns,
                      value=dataframe.columns[2])
dropdown_y = Dropdown(description='Y axis',
                      options=dataframe.columns,
                      value=dataframe.columns[3])

# Define callback function for dropdown widgets
def update_plot(evt):
    if evt is not None:
        new_value = evt['new']
        if evt['owner'] == dropdown_x:
            mark_scatter.x = dataframe[new_value]
            axis_x.label = new_value
        elif evt['owner'] == dropdown_y:
            mark_scatter.y = dataframe[new_value]
            axis_y.label = new_value


# Define callback function for selections
def on_select(evt):
    if evt is not None and evt['new'] is not None:
        indices = evt['new']
        datagrid.data = dataframe.iloc[indices].reset_index()

# Bind callback to the dropdown widgets
dropdown_x.observe(update_plot, names=['value'])
dropdown_y.observe(update_plot, names=['value'])
mark_scatter.observe(on_select, names=['selected'])

# Create datagrid
datagrid = bqw.DataGrid(data=dataframe, layout={'height': '300px'})

# Create Box containers
widget_box = HBox([dropdown_x, dropdown_y], layout={'margin': '10px'})
app_container = VBox([figure, widget_box, datagrid],
                     layout={'width':'100%'})

# Display the visualization
app_container
